In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import pymongo
import json

In [7]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.listings_db

# Declare the collection
collection = db.listings

def import_content(filepath):
    mng_client = pymongo.MongoClient('localhost', 27017)
    mng_db = mng_client['db'] // Replace mongo db name
    collection_name = 'collection' // Replace mongo db collection name
    db_cm = mng_db[collection_name]
    cdir = os.path.dirname(__file__)
    file_res = os.path.join(cdir, filepath)

    
    data = pd.read_csv(file_res,dtype=str,na_filter=False)
    data_json = json.loads(data.to_json(orient='records'))
    db_cm.remove()
    db_cm.insert(data_json)

if __name__ == "__main__":
  filepath = '/ADDL'  // pass csv file path
  import_content(filepath)

In [8]:
dc_listings = db.listings.find()

In [11]:
#dc_listings_df = pd.read_csv("dc_listings_clean.csv")

dc_listings_df.columns

Index(['id', 'accommodates', 'bedrooms', 'bathrooms', 'beds', 'price',
       'security_deposit', 'cleaning_fee', 'minimum_nights', 'maximum_nights',
       'number_of_reviews'],
      dtype='object')

In [19]:
dc_listings_df.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,160.0,1,1125,0
1,6,3.0,3.0,3.0,350.0,2,30,65
2,1,1.0,2.0,1.0,50.0,2,1125,1
3,2,1.0,1.0,1.0,95.0,1,1125,0
4,4,1.0,1.0,1.0,50.0,7,1125,0


In [12]:
dc_listings_df = dc_listings_df[['accommodates','bedrooms', 'bathrooms', 'beds', 'price','minimum_nights', 'maximum_nights','number_of_reviews']]
dc_listings_df.columns


Index(['accommodates', 'bedrooms', 'bathrooms', 'beds', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews'],
      dtype='object')

In [20]:
dc_listings_df.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,160.0,1,1125,0
1,6,3.0,3.0,3.0,350.0,2,30,65
2,1,1.0,2.0,1.0,50.0,2,1125,1
3,2,1.0,1.0,1.0,95.0,1,1125,0
4,4,1.0,1.0,1.0,50.0,7,1125,0


In [21]:
# Compare different Models : Models with different feature columns based on their RMSE value

In [22]:
# multivariate KNN model without using library
column_names_to_normalize = ['accommodates', 'bedrooms', 'bathrooms', 'beds','minimum_nights', 'maximum_nights', 'number_of_reviews']
df_minus_price = dc_listings_df[column_names_to_normalize]



In [23]:
train_feature1 = df_minus_price.copy()
train_feature2 = df_minus_price.copy()
train_feature3 = df_minus_price.copy()
#normalized_listings['price'] = dc_listings_df['price']

In [24]:
from sklearn.preprocessing import StandardScaler
import numpy as np
ss = StandardScaler()
normalized_listings = pd.DataFrame(ss.fit_transform(df_minus_price),columns = df_minus_price.columns)


/Users/mehrunnisha/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/mehrunnisha/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [25]:
normalized_listings['price'] = dc_listings_df['price']
normalized_listings = normalized_listings[['accommodates','bedrooms', 'bathrooms', 'beds', 'price','minimum_nights', 'maximum_nights','number_of_reviews']]

normalized_listings.head()


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.400108,-0.241412,-0.416947,0.305461,160.0,-0.345094,-0.016458,-0.516394
1,1.394171,2.133163,2.955773,1.150358,350.0,-0.069034,-0.016489,1.676470
2,-1.090986,-0.241412,1.269413,-0.539436,50.0,-0.069034,-0.016458,-0.482657
3,-0.593954,-0.241412,-0.416947,-0.539436,95.0,-0.345094,-0.016458,-0.516394
4,0.400108,-0.241412,-0.416947,-0.539436,50.0,1.311270,-0.016458,-0.516394


In [26]:
normalized_listings = normalized_listings.sample(frac=1,random_state=0)

norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

In [27]:
from scipy.spatial import distance

first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
first_fifth_distance

0.49703129931517454

In [30]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates']# 'bathrooms'
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

110.51996937433884


In [46]:
norm_test_df['price'].head()

1873     60.0
2614    100.0
3410     99.0
2138     75.0
798      70.0
Name: price, dtype: float64

In [31]:
norm_test_df['predicted_price'].head()

1873    115.8
2614    115.8
3410    115.8
2138     69.6
798     115.8
Name: predicted_price, dtype: float64

In [29]:
norm_test_df['predicted_price'].head()

1873    102.2
2614    102.2
3410    102.2
2138     86.6
798     102.2
Name: predicted_price, dtype: float64

In [22]:
# multivariate KNN model without using library

In [37]:
train_feature_df = train_feature1.copy().iloc[0:2792]
test_feature_df = train_feature1.copy().iloc[2792:]
train_target=dc_listings_df['price'].iloc[0:2792]
test_target=dc_listings_df['price'].iloc[2792:]



In [38]:
knn.fit(train_feature_df, train_target)
predictions = knn.predict(test_feature_df)

result = pd.DataFrame(predictions)


In [39]:
result.shape

(931, 1)

In [40]:
train_target=train_target.copy().iloc[0:931]

In [41]:
train_target.head()

0    160.0
1    350.0
2     50.0
3     95.0
4     50.0
Name: price, dtype: float64

In [42]:
result.head()

,0
0,91.0
1,110.2
2,166.0
3,150.2
4,153.6


In [111]:
#predictions

In [112]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(train_target, result)
two_features_rmse = two_features_mse ** (1/2)
print(two_features_rmse)


169.85250630777662


In [116]:
train_feature_df[cols].shape

(2792, 4)

In [117]:
train_target.shape

(931,)

In [43]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm='brute')


In [47]:
#Multiple Features
knn = KNeighborsRegressor(algorithm='brute')

cols = ['accommodates','bedrooms','bathrooms','beds']

knn.fit(train_feature_df[cols], train_target)
four_features_predictions = knn.predict(test_feature_df[cols])
four_features_mse = mean_squared_error(test_target, four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
four_features_rmse

In [48]:
#
prediction_df = pd.DataFrame(four_features_predictions)

In [158]:
prediction_df.head()

,0
0,113.2
1,147.0
2,179.4
3,235.6
4,257.4


In [170]:
#Multiple Features
knn = KNeighborsRegressor(n_neighbors=10,algorithm='brute')

cols = ['accommodates','bedrooms','bathrooms','beds','minimum_nights', 'maximum_nights','number_of_reviews']

knn.fit(train_feature_df[cols], train_target)
seven_features_predictions = knn.predict(test_feature_df[cols])
seven_features_mse = mean_squared_error(test_target, seven_features_predictions)
seven_features_rmse = seven_features_mse ** (1/2)
seven_features_rmse

117.02760748215259

In [ ]:
# multivariate KNN model with scikit-learn library

In [ ]:
# multivariate KNN model with scikit-learn library Ends

In [53]:
# Normalize Data and Run ML models again to predict prices

In [30]:
result = dc_listings.copy()

In [31]:
for feature_name in dc_listings.columns:
      max_value = dc_listings[feature_name].max()
      min_value = dc_listings[feature_name].min()
      result[feature_name] = (dc_listings[feature_name] - min_value) / (max_value - min_value)

In [32]:
result.head()

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
2645,0.133333,0.1,0.125,0.0625,0.023115,0.100000,0.1250,0.03352,8.335337e-08,0.066298,0.0
2825,0.133333,0.3,0.250,0.1250,0.039118,0.000000,0.0000,0.00000,5.234033e-07,0.000000,0.0
2145,0.133333,0.1,0.250,0.1250,0.028450,0.000000,0.0000,0.00000,5.234033e-07,0.151934,0.0
2541,0.133333,0.1,0.125,0.0625,0.014225,0.000000,0.0625,0.00000,5.234033e-07,0.002762,0.0
3349,0.133333,0.1,0.125,0.0625,0.033784,0.033333,0.1250,0.00000,5.234033e-07,0.019337,0.0


In [33]:
def normalize(x):
    try:
        x = x/np.linalg.norm(x,ord=1)
        return x
    except :
        raise
dc_listings_norm = pd.DataFrame.apply(dc_listings,normalize)

In [34]:
dc_listings_norm.head()

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
2645,0.000252,0.000223,0.000215,0.000164,0.000135,0.000744,0.000359,0.000836,8.331472e-08,0.000421,0.0
2825,0.000252,0.000670,0.000431,0.000328,0.000216,0.000000,0.000000,0.000119,5.207170e-07,0.000000,0.0
2145,0.000252,0.000223,0.000431,0.000328,0.000162,0.000000,0.000000,0.000119,5.207170e-07,0.000965,0.0
2541,0.000252,0.000223,0.000215,0.000164,0.000090,0.000000,0.000179,0.000119,5.207170e-07,0.000018,0.0
3349,0.000252,0.000223,0.000215,0.000164,0.000189,0.000248,0.000359,0.000119,5.207170e-07,0.000123,0.0


In [35]:
import sklearn.preprocessing   
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
df_scaled = pd.DataFrame(ss.fit_transform(dc_listings),columns = dc_listings.columns)

/Users/mehrunnisha/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/mehrunnisha/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [36]:
df_scaled.head()

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
0,-0.096923,-0.241412,-0.416947,-0.539436,-0.529410,0.889191,0.271113,1.311270,-0.016485,0.293279,-0.992754
1,-0.096923,2.133163,1.269413,0.305461,-0.208192,-0.502428,-0.809242,-0.345094,-0.016458,-0.516394,-0.992754
2,-0.096923,-0.241412,1.269413,0.305461,-0.422337,-0.502428,-0.809242,-0.345094,-0.016458,1.339106,-0.992754
3,-0.096923,-0.241412,-0.416947,-0.539436,-0.707864,-0.502428,-0.269064,-0.345094,-0.016458,-0.482657,-0.992754
4,-0.096923,-0.241412,-0.416947,-0.539436,-0.315264,-0.038555,0.271113,-0.345094,-0.016458,-0.280239,-0.992754


In [39]:
import pandas as pd
from sklearn import preprocessing

In [41]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(dc_listings)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.head()

/Users/mehrunnisha/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1,2,3,4,5,6,7,8,9,10
0,0.133333,0.1,0.125,0.0625,0.023115,0.100000,0.1250,0.03352,8.335337e-08,0.066298,0.0
1,0.133333,0.3,0.250,0.1250,0.039118,0.000000,0.0000,0.00000,5.234033e-07,0.000000,0.0
2,0.133333,0.1,0.250,0.1250,0.028450,0.000000,0.0000,0.00000,5.234033e-07,0.151934,0.0
3,0.133333,0.1,0.125,0.0625,0.014225,0.000000,0.0625,0.00000,5.234033e-07,0.002762,0.0
4,0.133333,0.1,0.125,0.0625,0.033784,0.033333,0.1250,0.00000,5.234033e-07,0.019337,0.0


In [42]:
data = dc_listings.copy()
dataf=((data-data.min())/(data.max()-data.min()))*20

In [43]:
dataf.sample(5)

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
3223,6.666667,4.0,2.5,3.75,1.173542,2.000000,7.50,0.335196,3.390014e-06,0.000000,4.615385
3132,1.333333,2.0,2.5,1.25,0.320057,0.000000,0.50,0.000000,5.587935e-08,0.883978,1.538462
1551,1.333333,2.0,2.5,1.25,0.227596,1.666667,0.00,0.335196,1.046807e-05,0.828729,1.538462
3129,6.666667,4.0,5.0,5.00,1.173542,1.333333,6.75,0.223464,5.587935e-08,0.000000,4.615385
1187,0.000000,2.0,2.5,1.25,0.867710,0.000000,3.75,0.446927,1.046807e-05,0.000000,3.076923


In [44]:
dataf.drop('distance',axis=1)

train_df = dataf.copy().iloc[:2792]
test_df = dataf.copy().iloc[2792:]

In [45]:
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dataf[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

In [46]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')

In [47]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

1.5148597416110618

In [48]:
test_df['predicted_price'].head()

2850    0.523471
2279    0.523471
2771    2.349929
910     2.349929
2434    2.349929
Name: predicted_price, dtype: float64

In [61]:
dc_listings.drop('distance',axis=1)

train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

In [62]:
def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

In [63]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column='accommodates')

In [64]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

212.98927967051543

In [53]:
test_df['predicted_price'].head()

2850     83.6
2279     83.6
2771    340.4
910     340.4
2434    340.4
Name: predicted_price, dtype: float64

In [58]:
normalized_listings = dc_listings.sample(frac=1,random_state=0)
normalized_listings.drop('distance',axis=1)
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

In [59]:
norm_test_df.head()

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
3620,2,1.0,1.0,2.0,80.0,300.0,0.0,4,1125,3,1
3132,2,1.0,1.0,1.0,55.0,0.0,10.0,1,7,16,1
3210,6,2.0,1.0,5.0,175.0,100.0,100.0,1,1125,11,3
3523,2,1.0,1.0,2.0,120.0,0.0,30.0,5,1125,8,1
2802,4,1.0,1.0,2.0,150.0,0.0,75.0,4,1125,3,1


In [60]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

NameError: ("name 'distance' is not defined", 'occurred at index 3620')

In [37]:
#from sklearn import preprocesing
#import sklearn.preprocessing
#from sklearn import preprocesing
#x = pd.concat([dc_listings.accommodates, dc_listings.bedrooms,dc_listings.bathrooms	])
#min_max_scaler = sklearn.preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#x_new = pd.DataFrame(x_scaled)
#dc_listings_norm2 = x_new


#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num

In [47]:
#Normalize the dataset
normalized_listings = df_scaled.sample(frac=1,random_state=0)
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]

In [48]:
normalized_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,distance
2645,-0.593954,-0.241412,2.112593,-0.539436,-0.172501,0.889191,0.271113,1.311270,-0.016489,-0.482657,-0.295654
1503,0.400108,-1.428699,-0.416947,-0.539436,-0.350955,1.816937,0.271113,0.483088,-0.016485,-0.482657,-0.295654
1791,-0.593954,-0.241412,-0.416947,-0.539436,-0.172501,-0.502428,-0.269064,-0.345094,-0.016458,-0.516394,-0.295654
3539,1.891202,2.133163,2.112593,1.995255,0.077335,0.193381,0.811291,-0.069034,-0.016490,-0.482657,1.795645
489,0.400108,-0.241412,-0.416947,0.305461,-0.029738,1.816937,0.271113,0.207027,-0.016458,-0.347712,-0.295654


In [71]:

#Normalize the dataset 2
normalized_listings = ny_listings_test3.sample(frac=1,random_state=0)
norm_train_df = normalized_listings.copy().iloc[0:4200]
norm_test_df = normalized_listings.copy().iloc[4200:]

NameError: name 'ny_listings_test3' is not defined

In [49]:
from scipy.spatial import distance

first_listing = normalized_listings.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = normalized_listings.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
first_fifth_distance

0.8431799640557058

In [ ]:
def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

In [ ]:
#norm_test_df['predicted_price']

In [ ]:
#preprocessing.normalize(data)

In [ ]:
import sklearn.preprocessing   
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
df_scaled = pd.DataFrame(ss.fit_transform(dc_listings),columns = dc_listings.columns)

In [1]:
dc_listings.shape

NameError: name 'dc_listings' is not defined

In [ ]:
predicted_price = [113.6,134.6,191.0,75.0,161.0]
KNN1_result_df['predicted_price'] = predicted_price
test_feature_df.head()

KNN1_result_df = dc_listings_df.iloc[2792:2797,:]
predicted_price = [113.6,134.6,191.0,75.0,161.0]
KNN1_result_df['predicted_price'] = predicted_price
KNN1_result_df